In [1]:
import os
#execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

In [3]:
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.enableHiveSupport().master("local").getOrCreate()

In [4]:
graphPath = "/data/graphDFSample"

In [5]:
from pyspark.sql.functions import udf
import pyspark.sql.types as t

def emit_all_pairs(friends_array):
    ret = []
    friends_len = len(friends_array)
    for i in range(friends_len):
        for j in range(i+1,friends_len):
            ret.append((friends_array[i], friends_array[j]))
    return ret

pair_schema = t.StructType([
    t.StructField("u1", t.IntegerType(), False),
    t.StructField("u2", t.IntegerType(), False)
])

emit_all_pairs_udf = udf(emit_all_pairs, t.ArrayType(pair_schema))

In [6]:
from pyspark.sql.functions import explode, collect_list, size, col, row_number, sort_array, count, desc, size
from pyspark.sql import Window

reversedGraph = sparkSession.read.parquet(graphPath) \
    .withColumn("friend", explode('friends')) \
    .groupBy("friend") \
    .agg(collect_list("user").alias("users")) \
    .withColumn("users", sort_array('users')) \
    .where(size("users") > 1)
                
    
pairs = reversedGraph.withColumn("pairs", emit_all_pairs_udf("users")) \
    .withColumn("pair", explode("pairs")) \
    .groupBy("pair") \
    .agg(count("pair").alias("pair_count"))

In [ ]:
results = pairs.select(col("pair_count"), "pair.*") \
           .orderBy(desc("pair_count"), desc("u1"), desc("u2")) \
           .limit(49) \
          .collect()

In [8]:
for pair, u1, u2 in results:
    print("{} {} {}".format(pair, u1, u2))

3206 27967558 42973992
3130 20158643 42973992
3066 22582764 42973992
3044 21864412 51640390
3021 17139850 51640390
3010 14985079 51640390
2970 17139850 21864412
2913 20158643 27967558
2903 22280814 51151280
2870 23848749 51640390
2855 20158643 22582764
2849 20158643 44996025
2846 22280814 42973992
2784 21864412 23848749
2779 31964081 51640390
2776 39205988 51640390
2754 17139850 23848749
2749 22582764 27967558
2728 50561859 51640390
2724 15485897 51640390
2700 28135661 42973992
2655 22280814 27967558
2653 42973992 43548989
2639 26755857 51640390
2621 14635589 51640390
2608 15485897 17139850
2606 17139850 26755857
2601 21864412 39205988
2600 8406745 51640390
2599 37735419 51640390
2597 20158643 28135661
2585 40003405 42973992
2585 21864412 31964081
2581 27967558 43548989
2579 23848749 31964081
2578 27967558 28135661
2578 15485897 21864412
2577 42973992 64755069
2574 51151280 57077210
2573 20158643 43548989
2566 21864412 26755857
2564 22280814 64755069
2561 42973992 44996025
2556 1713985